# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [229]:
# import libraries

import pandas as pd
import numpy as np
from sqlalchemy import create_engine

import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

import sklearn
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics.scorer import make_scorer

import pickle

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\livel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\livel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\livel\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///database.db')
df = pd.read_sql_table('data',con=engine)

In [3]:
print(df.head())

                                             message  \
0  Weather update - a cold front from Cuba that c...   
1            Is the Hurricane over or is it not over   
2                    Looking for someone but no name   
3  UN reports Leogane 80-90 destroyed. Only Hospi...   
4  says: west side of Haiti, rest of the country ...   

                                            original   genre  related  \
0  Un front froid se retrouve sur Cuba ce matin. ...  direct        1   
1                 Cyclone nan fini osinon li pa fini  direct        1   
2  Patnm, di Maryani relem pou li banm nouvel li ...  direct        1   
3  UN reports Leogane 80-90 destroyed. Only Hospi...  direct        1   
4  facade ouest d Haiti et le reste du pays aujou...  direct        1   

   request  offer  aid_related  medical_help  medical_products  \
0        0      0            0             0                 0   
1        0      0            1             0                 0   
2        0      0         

In [4]:
X = df.iloc[:,0]
Y = df.iloc[:,3:]

In [31]:
print(Y)

       related  request  offer  aid_related  medical_help  medical_products  \
0            1        0      0            0             0                 0   
1            1        0      0            1             0                 0   
2            1        0      0            0             0                 0   
3            1        1      0            1             0                 1   
4            1        0      0            0             0                 0   
...        ...      ...    ...          ...           ...               ...   
26210        0        0      0            0             0                 0   
26211        0        0      0            0             0                 0   
26212        1        0      0            0             0                 0   
26213        1        0      0            1             0                 0   
26214        1        0      0            0             0                 0   

       search_and_rescue  security  military  child

In [188]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

### 2. Write a tokenization function to process your text data

In [9]:
def tokenize(text):
    
    text = re.sub(r"[^a-zA-Z0-9]", " ",text).lower()
    text = word_tokenize(text)
    text =  [w for w in text if w not in stopwords.words("english")]
    text = [PorterStemmer().stem(w) for w in text]
    text = [WordNetLemmatizer().lemmatize(w) for w in text]
    
    return text

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [10]:
'''
    normalized = text.apply(lambda x: re.sub(r"[^a-zA-Z0-9]", " ",x))
    normalized = normalized.apply(lambda x: x.lower())
    tokenized = tokenize(normalized)
    stop_remove = tokenized.apply(lambda x: [w for w in x if w not in stopwords.words("english")])
    stemmed = stop_remove.apply(lambda x: [PorterStemmer().stem(w) for w in x])
    lemmed = stemmed.apply(lambda x: [WordNetLemmatizer().lemmatize(w) for w in x])
  
    return lemmed
'''

'\n  normalized = text.apply(lambda x: re.sub(r"[^a-zA-Z0-9]", " ",x))\n  normalized = normalized.apply(lambda x: x.lower())\n  tokenized = tokenize(normalized)\n  stop_remove = tokenized.apply(lambda x: [w for w in x if w not in stopwords.words("english")])\n  stemmed = stop_remove.apply(lambda x: [PorterStemmer().stem(w) for w in x])\n  lemmed = stemmed.apply(lambda x: [WordNetLemmatizer().lemmatize(w) for w in x])\n\n  \n  return lemmed\n  '

In [186]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('ifidf', TfidfTransformer()),
    ('clf', RandomForestClassifier())
])

In [180]:
'''
def preprocess(X):
    
    vectorizer = CountVectorizer(tokenizer=tokenize)
    X = vectorizer.fit_transform(X)
    tfidf = TfidfTransformer()
    X = tfidf.fit_transform(X)
    
    return X
'''

In [183]:
'''
def train(X, y):
    
    clfs = [] 
    
    num_col = y.shape[1]
    
    print('num_col:', num_col)
   
    for i in range(num_col):
 
        print('i:', i)
        clf = LogisticRegression(class_weight='balanced')
        clf.fit(X, y[:,i])
    
        clfs.append(clf)
    
    return clfs 
'''

In [184]:
'''
def pipeline_orig(X, y): 
    
    X = preprocess(X)
    
    clfs = train(X, y, classifier)
    
    return clfs
'''

'\ndef pipeline_orig(X, y): \n    \n    X = preprocess(X)\n    \n    clfs = train(X, y, classifier)\n    \n    return clfs\n'

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [192]:
print(X_train.values)

['I am not a professional but have experience with tools . Hand tools and power tools'
 "Hurricane Recovery. (@Pig 'n' Whistle w/ 9 others) http://t.co/HcXZLzcGWalked to the MET.. But Sandy kept it closed =[ @Metropolitan Museum Steps http://t.co/MCZ9rVgS"
 'Patricia tresil from Mirebalais, Barcelona Fanatical. ' ...
 'Visibility is low because of the thick smog and we could hear volcanic tremors," he told AFP.'
 'ISLAMABAD, 1 September 2009 (IRIN) - Dozens of people who suffered spinal cord injuries in the October 2005 earthquake in northern Pakistan are still seeking suitable accommodation and an opportunity to rebuild lives.'
 'In other areas, as soon as the chopper lands, everyone rushes there."']


In [201]:
pipeline.fit(X_train, y_train.iloc[:,0].values)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                               

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [202]:
y_pred = pipeline.predict(X_test)

In [203]:
y_test = y_test.values
print(y_test)

AttributeError: 'numpy.ndarray' object has no attribute 'values'

In [204]:
label_pred = y_pred
print(label_pred)
print(label_pred.sum(), len(label_pred))

[0 1 1 ... 1 1 1]
5752 6554


In [209]:
def display_results(y_test, y_pred):
    
    try: 
        num_col = num_col = y_pred.shape[1]
    except: 
        num_col = 1
        
    labels = Y.columns
    
    accuracies = [] 
    fvalues = []
    
    for i in range(num_col):
        
        try:
            pred = y_pred[:,i]
            test = y_test[:,i]
        except:
            pred = y_pred
            test = y_test
        
        tp = np.logical_and(test, pred).sum()
        fp = np.logical_and(1-test, pred).sum()
        tn = np.logical_and(1-test, 1-pred).sum()
        fn = np.logical_and(test, 1-pred).sum()

        acc = (tp + tn) / (tp + fp + tn + fn)
    
        recall = tp / (tp + fn)
        prec = tp / (tp + fp)
        fval = 2 * recall * prec / (recall + prec)
                
        conf_matrix = [[tp, fp],[fn, tn]]
    
        print('label:', labels[i] ,'\naccuracy:', acc, '\nf-value:', fval)
        print('confusion matrix:')
        print(conf_matrix[0])
        print(conf_matrix[1])
        
        print('\n\n')
        
        accuracies.append(acc)
        fvalues.append(fval)
        
    
    print('average accuracy:', sum(accuracies) / len(accuracies))
    
    fvalues = [x for x in fvalues if x == x]
    
    ave_fvalues = sum(fvalues) / len(fvalues)
    
    print('average f score:', ave_fvalues)
    #print("Labels:", labels)
    #print("Accuracy:", accuracy)
    #print("Confusion Matrix:\n", confusion_mat)
    
    return ave_fvalues

In [210]:
print(display_results(y_test[:,0], y_pred))

label: related 
accuracy: 0.814206834800776 
f-value: 0.8845832947065912
confusion matrix:
[4771, 955]
[290, 685]



average accuracy: 0.814206834800776
average f score: 0.8845832947065912
0.8845832947065912


### 6. Improve your model
Use grid search to find better parameters. 

In [213]:
print(sorted(sklearn.metrics.SCORERS.keys()))

['accuracy', 'adjusted_mutual_info_score', 'adjusted_rand_score', 'average_precision', 'balanced_accuracy', 'completeness_score', 'explained_variance', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'fowlkes_mallows_score', 'homogeneity_score', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_weighted', 'max_error', 'mutual_info_score', 'neg_brier_score', 'neg_log_loss', 'neg_mean_absolute_error', 'neg_mean_gamma_deviance', 'neg_mean_poisson_deviance', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_median_absolute_error', 'neg_root_mean_squared_error', 'normalized_mutual_info_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'r2', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'roc_auc', 'roc_auc_ovo', 'roc_auc_ovo_weighted', 'roc_auc_ovr', 'roc_auc_ovr_weighted', 'v_measure_score']


In [222]:
parameters = {
    'clf__max_depth': [10],
    'clf__n_estimators': [200],
    'clf__min_samples_leaf': [1, 2, 5, 10]
    }

my_scorer = make_scorer(display_results, greater_is_better=True)

CV = GridSearchCV(pipeline, parameters, scoring=my_scorer, n_jobs= 1)

In [223]:
print(X_train.head())

11046    I am not a professional but have experience wi...
12798    Hurricane Recovery. (@Pig 'n' Whistle w/ 9 oth...
8749     Patricia tresil from Mirebalais, Barcelona Fan...
15771    Over 230 specialist eye care nurses and occupa...
25107    Food riots erupted in Bhubaneswar, the state c...
Name: message, dtype: object


In [224]:
CV.fit(X_train, y_train.iloc[:,0].values)

label: related 
accuracy: 0.7616763443574854 
f-value: 0.8647176841501862
confusion matrix:
[3017, 944]
[0, 0]



average accuracy: 0.7616763443574854
average f score: 0.8647176841501862
label: related 
accuracy: 0.7618085375094721 
f-value: 0.8648028673835125
confusion matrix:
[3016, 943]
[0, 0]



average accuracy: 0.7618085375094721
average f score: 0.8648028673835125
label: related 
accuracy: 0.7618085375094721 
f-value: 0.8648028673835125
confusion matrix:
[3016, 943]
[0, 0]



average accuracy: 0.7618085375094721
average f score: 0.8648028673835125
label: related 
accuracy: 0.7616161616161616 
f-value: 0.8646788990825688
confusion matrix:
[3016, 944]
[0, 0]



average accuracy: 0.7616161616161616
average f score: 0.8646788990825688
label: related 
accuracy: 0.7616161616161616 
f-value: 0.8646788990825688
confusion matrix:
[3016, 944]
[0, 0]



average accuracy: 0.7616161616161616
average f score: 0.8646788990825688
label: related 
accuracy: 0.7616763443574854 
f-value: 0.86471768

KeyboardInterrupt: 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [110]:
y_pred_improved = CV.predict(X_test.values)

AttributeError: 'GridSearchCV' object has no attribute 'best_estimator_'

In [ ]:
display_results(y_test[:,0], y_pred_improved)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [225]:
pipelineLR = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('ifidf', TfidfTransformer()),
    ('clf', LogisticRegression(class_weight='balanced'))
])

In [226]:
pipelineLR.fit(X_train.values, y_train.iloc[:,0].values)

C:\Users\livel\Miniconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                ('ifidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                                    fit_intercept=True, intercept_scaling

In [118]:
y_pred_improved2 = pipelineLR.predict(X_test.values)

In [227]:
print(display_results(y_test[:,0], y_pred_improved2))

label: related 
accuracy: 0.5943183526228769 
f-value: 0.7201078502540703
confusion matrix:
[3472, 1122]
[1577, 482]



average accuracy: 0.5943183526228769
average f score: 0.7201078502540703
0.7201078502540703


### 9. Export your model as a pickle file

In [232]:
filename = 'finalized_model.pkl'
pickle.dump(pipeline, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.